# Train 1D CNN

1D Convolution을 기본 구성 요소로 하는 EEG classifier를 학습해보는 노트북.

- Three-way SoftMax classifier of normal, non-vascular MCI, and non-vascular dementia

-----

## 환경 구성

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# Load some packages
import os
import glob
import json
import datetime
from copy import deepcopy

import matplotlib.pyplot as plt
import pprint
from IPython.display import clear_output
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
from tqdm.auto import tqdm 

import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from typing import Type, Any, Callable, Union, List, Optional
from itertools import cycle

from torch.utils.tensorboard import SummaryWriter
import wandb

# custom package
from utils.eeg_dataset import *

In [3]:
# notebook name
def get_notebook_name():
    import ipynbname
    return ipynbname.name()
nb_fname = get_notebook_name()

In [4]:
# Other settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # cleaner text

plt.style.use('default') 
# ['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 
#  'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 
#  'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 
#  'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 
#  'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']

plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams["font.family"] = 'NanumGothic' # for Hangul in Windows

In [5]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.9.0
cuda is available.


In [6]:
# Data file path
data_path = r'dataset/02_Curated_Data/'
meta_path = os.path.join(data_path, 'metadata_debug.json')

with open(meta_path, 'r') as json_file:
    metadata = json.load(json_file)

pprint.pprint(metadata[0])

{'age': 78,
 'birth': '1940-06-02',
 'dx1': 'mci_rf',
 'edfname': '00001809_261018',
 'events': [[0, 'Start Recording'],
            [0, 'New Montage - Montage 002'],
            [36396, 'Eyes Open'],
            [72518, 'Eyes Closed'],
            [73862, 'Eyes Open'],
            [75248, 'Eyes Closed'],
            [76728, 'swallowing'],
            [77978, 'Eyes Open'],
            [79406, 'Eyes Closed'],
            [79996, 'Photic On - 3.0 Hz'],
            [80288, 'Eyes Open'],
            [81296, 'Eyes Closed'],
            [82054, 'Photic Off'],
            [84070, 'Photic On - 6.0 Hz'],
            [84488, 'Eyes Open'],
            [85538, 'Eyes Closed'],
            [86086, 'Photic Off'],
            [88144, 'Photic On - 9.0 Hz'],
            [90160, 'Photic Off'],
            [91458, 'Eyes Open'],
            [92218, 'Photic On - 12.0 Hz'],
            [92762, 'Eyes Closed'],
            [94198, 'Photic Off'],
            [94742, 'Eyes Open'],
            [95708, 'Eyes Close

-----

## Data Filtering by Diagnosis

#### Non-Vascular Dementia, Non-Vascular MCI, Normal

In [7]:
diagnosis_filter = [
    # Normal
    {'type': 'Normal',
     'include': ['normal'], 
     'exclude': []},
    # Non-vascular MCI
    {'type': 'Non-vascular MCI',
     'include': ['mci'], 
     'exclude': ['mci_vascular']},
    # Non-vascular dementia
    {'type': 'Non-vascular dementia',
     'include': ['dementia'], 
     'exclude': ['vd']},
]

def generate_class_label(label):
    for c, f in enumerate(diagnosis_filter):
        inc = set(f['include']) & set(label) == set(f['include'])
        # inc = len(set(f['include']) & set(label)) > 0        
        exc = len(set(f['exclude']) & set(label)) == 0
        if  inc and exc:
            return (c, f['type'])
    return (-1, 'The others')

class_label_to_type = [d_f['type'] for d_f in diagnosis_filter]
print('class_label_to_type:', class_label_to_type)

class_label_to_type: ['Normal', 'Non-vascular MCI', 'Non-vascular dementia']


In [8]:
splitted_metadata = [[] for i in diagnosis_filter]

for m in metadata:
    c, n = generate_class_label(m['label'])
    if c >= 0:
        m['class_type'] = n
        m['class_label'] = c
        splitted_metadata[c].append(m)
        
for i, split in enumerate(splitted_metadata):
    if len(split) == 0:
        print(f'(Warning) Split group {i} has no data.')
    else:
        print(f'- There are {len(split):} data belonging to {split[0]["class_type"]}')

- There are 463 data belonging to Normal
- There are 347 data belonging to Non-vascular MCI
- There are 229 data belonging to Non-vascular dementia


-----

## Configure the Train, Validation, and Test Splits

#### Split the filtered dataset and shuffle them

In [9]:
# random seed
random.seed(0)

# Train : Val : Test = 8 : 1 : 1
ratio1 = 0.8
ratio2 = 0.1

metadata_train = []
metadata_val = []
metadata_test = []

for split in splitted_metadata:
    random.shuffle(split)
    
    n1 = round(len(split) * ratio1)
    n2 = n1 + round(len(split) * ratio2)

    metadata_train.extend(split[:n1])
    metadata_val.extend(split[n1:n2])
    metadata_test.extend(split[n2:])

random.shuffle(metadata_train)
random.shuffle(metadata_val)
random.shuffle(metadata_test)

print('Train data size\t\t:', len(metadata_train))
print('Validation data size\t:', len(metadata_val))
print('Test data size\t\t:', len(metadata_test))

print('\n', '--- Recheck ---', '\n')
train_class_nums = np.zeros((len(class_label_to_type)), dtype=np.int32)
for m in metadata_train:
    train_class_nums[m['class_label']] += 1

val_class_nums = np.zeros((len(class_label_to_type)), dtype=np.int32)
for m in metadata_val:
    val_class_nums[m['class_label']] += 1

test_class_nums = np.zeros((len(class_label_to_type)), dtype=np.int32)
for m in metadata_test:
    test_class_nums[m['class_label']] += 1

print('Train data label distribution\t:', train_class_nums, train_class_nums.sum())
print('Val data label distribution\t:', val_class_nums, val_class_nums.sum())
print('Test data label distribution\t:', test_class_nums, test_class_nums.sum())

# random seed
random.seed()

# print([m['serial']  for m in metadata_train[:15]])
# print([m['serial']  for m in metadata_val[:15]])
# print([m['serial']  for m in metadata_test[:15]])

Train data size		: 831
Validation data size	: 104
Test data size		: 104

 --- Recheck --- 

Train data label distribution	: [370 278 183] 831
Val data label distribution	: [46 35 23] 104
Test data label distribution	: [47 34 23] 104


#### Wrap the splitted data using PyTorch Dataset

In [10]:
ages = []
for m in metadata_train:
    ages.append(m['age'])

ages = np.array(ages)
age_mean = np.mean(ages)
age_std = np.std(ages)

print('Age mean and standard deviation:')
print(age_mean, age_std)

Age mean and standard deviation:
69.92779783393502 9.817569889945597


In [11]:
crop_length = 200 * 10 # 10 seconds
awgn_mean = 0.0
awgn_std = 3e-1

# composed = transforms.Compose([EEGRandomCrop(crop_length=crop_length)])
# train_dataset = EEGDataset(data_path, metadata_train, composed)

# signal_means = []
# signal_stds = []

# for i in range(10):
#     for d in train_dataset:
#         signal_means.append(d['signal'].mean(axis=1, keepdims=True))
#         signal_stds.append(d['signal'].std(axis=1, keepdims=True))
        
# signal_means = np.mean(np.array(signal_means), axis=0)
# signal_stds = np.mean(np.array(signal_stds), axis=0)

# print('Mean and standard deviation for signal:')
# print(signal_means, '\n\n', signal_stds)

signal_means = np.array([[ 0.1127599 ], [ 0.06298441], [-0.02522413], [ 0.00508518], 
                         [ 0.12026667], [-0.19987741], [-0.00516898], [ 0.00239212], 
                         [-0.02861219], [-0.02973673], [-0.02515898], [-0.00060568], 
                         [ 0.04921601], [-0.00562142], [-0.04888308], [-0.0438447 ], 
                         [ 0.07532331], [-0.01890181], [-0.044876  ], [-0.00365138], [-0.01564376]])
signal_stds = np.array([[46.09896  ], [20.50783  ], [11.196733 ], [11.236944 ], [15.070532 ], 
                        [47.664406 ], [19.32747  ], [10.106162 ], [11.314243 ], [15.065008 ],
                        [20.478817 ], [13.86243  ], [13.2378435], [21.554531 ], [16.875841 ],
                        [13.989367 ], [19.789454 ], [10.839711 ], [11.179158 ], [94.12114  ], [65.64865  ]])

composed_train = transforms.Compose([EEGNormalizeAge(mean=age_mean, std=age_std),
                                     EEGRandomCrop(crop_length=crop_length),
                                     EEGNormalizeMeanStd(mean=signal_means, std=signal_stds),
                                     EEGDropPhoticChannel(),
                                     EEGAddGaussianNoise(mean=awgn_mean, std=awgn_std),
                                     EEGToTensor()])

composed_test = transforms.Compose([EEGNormalizeAge(mean=age_mean, std=age_std),
                                    EEGRandomCrop(crop_length=crop_length),
                                    EEGNormalizeMeanStd(mean=signal_means, std=signal_stds),
                                    EEGDropPhoticChannel(),
                                    EEGToTensor()])

longer_composed_test = transforms.Compose([EEGNormalizeAge(mean=age_mean, std=age_std),
                                           EEGRandomCrop(crop_length=crop_length * 10),
                                           EEGNormalizeMeanStd(mean=signal_means, std=signal_stds),
                                           EEGDropPhoticChannel(),EEGToTensor()])

train_dataset = EEGDataset(data_path, metadata_train, composed_train)
val_dataset = EEGDataset(data_path, metadata_val, composed_test)
test_dataset = EEGDataset(data_path, metadata_test, composed_test)
longer_test_dataset = EEGDataset(data_path, metadata_test, composed_test)

print(train_dataset[0]['signal'].shape)
print(train_dataset[0])

print()
print('-' * 100)
print()

print(val_dataset[0]['signal'].shape)
print(val_dataset[0])

print()
print('-' * 100)
print()

print(test_dataset[0]['signal'].shape)
print(test_dataset[0])

print()
print('-' * 100)
print()

print(longer_test_dataset[0]['signal'].shape)
print(longer_test_dataset[0])

torch.Size([20, 2000])
{'signal': tensor([[ 0.7583,  0.7835,  0.4607,  ..., -0.0212,  0.2512, -0.2997],
        [ 0.1433, -0.7037, -0.4211,  ...,  0.0729,  0.3907,  0.3185],
        [ 3.3368,  4.2569,  3.8936,  ...,  0.7546,  0.9782,  0.3295],
        ...,
        [ 0.4901,  0.2351,  0.4855,  ..., -0.1136,  0.6410,  0.6906],
        [-1.3538, -1.1217, -0.9359,  ...,  0.7194,  0.6806,  0.0356],
        [-0.0848, -0.5146, -0.3416,  ..., -0.0712, -0.2681, -1.0080]]), 'age': tensor(-1.2149), 'class_label': tensor(0), 'metadata': {'serial': '01012', 'edfname': '01212635_270515', 'birth': '1956-06-01', 'record': '2015-05-27T09:37:24', 'age': 58, 'dx1': 'cb_normal', 'label': ['normal', 'cb_normal'], 'events': [[0, 'Start Recording'], [0, 'New Montage - Montage 002'], [400, 'Eyes Open'], [7918, 'Eyes Closed'], [14091, 'Eyes Open'], [18208, 'Eyes Closed'], [24256, 'Eyes Open'], [30724, 'Eyes Closed'], [36562, 'Eyes Open'], [42190, 'Eyes Closed'], [48910, 'Eyes Open'], [55126, 'Eyes Closed'], [6

#### Data loader test

In [12]:
print('Current PyTorch device:', device)
if device.type == 'cuda':
    num_workers = 0 # A number other than 0 causes an error
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = DataLoader(train_dataset, 
                          batch_size=32, 
                          shuffle=True, 
                          drop_last=True,
                          num_workers=num_workers, 
                          pin_memory=pin_memory,
                          collate_fn=eeg_collate_fn)

for i_batch, sample_batched in enumerate(train_loader):
    sample_batched['signal'].to(device)
    sample_batched['age'].to(device)
    sample_batched['class_label'].to(device)
    
    print(i_batch, 
          sample_batched['signal'].shape, 
          sample_batched['age'].shape, 
          sample_batched['class_label'].shape, 
          len(sample_batched['metadata']))
    
    if i_batch > 3:
        break

Current PyTorch device: cuda
0 torch.Size([32, 20, 2000]) torch.Size([32]) torch.Size([32]) 32
1 torch.Size([32, 20, 2000]) torch.Size([32]) torch.Size([32]) 32
2 torch.Size([32, 20, 2000]) torch.Size([32]) torch.Size([32]) 32
3 torch.Size([32, 20, 2000]) torch.Size([32]) torch.Size([32]) 32
4 torch.Size([32, 20, 2000]) torch.Size([32]) torch.Size([32]) 32


#### Train, validation, test dataloaders

In [13]:
train_loader = DataLoader(train_dataset, 
                          batch_size=32, 
                          shuffle=True, 
                          drop_last=True,
                          num_workers=num_workers, 
                          pin_memory=pin_memory,
                          collate_fn=eeg_collate_fn)

val_loader = DataLoader(val_dataset, 
                        batch_size=32, 
                        shuffle=False, 
                        drop_last=False,
                        num_workers=num_workers, 
                        pin_memory=pin_memory,
                        collate_fn=eeg_collate_fn)

test_loader = DataLoader(test_dataset, 
                         batch_size=32, 
                         shuffle=False, 
                         drop_last=False,
                         num_workers=num_workers, 
                         pin_memory=pin_memory,
                         collate_fn=eeg_collate_fn)

longer_test_loader = DataLoader(test_dataset, 
                         batch_size=16, 
                         shuffle=False, 
                         drop_last=False,
                         num_workers=num_workers, 
                         pin_memory=pin_memory,
                         collate_fn=eeg_collate_fn)

-----

## Define 1D CNN models

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def print_final_shape(model):
    x = torch.zeros_like(sample_batched['signal']).to(device)
    model(x, age=sample_batched['age'].to(device), print_shape=True)


def visualize_network_tensorboard(model, name):
    # default `log_dir` is "runs" - we'll be more specific here
    writer = SummaryWriter('runs/' + nb_fname + '_' + name)

    for batch_i, sample_batched in enumerate(train_loader):
        # pull up the batch data
        x = sample_batched['signal'].to(device)
        age = sample_batched['age'].to(device)
        target = sample_batched['class_label'].to(device)

        # apply model on whole batch directly on device
        writer.add_graph(model, (x, age))
        output = model(x, age, print_shape=True)
        break
        
    writer.close()

#### 1D Tiny CNN

In [15]:
class TinyCNN(nn.Module):
    def __init__(self, n_input=20, n_output=3, stride=7, n_channel=64, 
                 use_age=True, final_pool='max'):
        super().__init__()
        
        if final_pool not in {'average', 'max'}:
            raise ValueError("final_pool must be set to one of ['average', 'max']")
        
        self.use_age = use_age
        
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=35, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=7)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(2)
        
        if final_pool == 'average':
            self.final_pool = nn.AdaptiveAvgPool1d(1)
        elif final_pool == 'max':
            self.final_pool = nn.AdaptiveMaxPool1d(1)
        
        if self.use_age:        
            self.fc1 = nn.Linear(n_channel + 1, n_channel)
        else:
            self.fc1 = nn.Linear(n_channel, n_channel)
            
        self.dropout = nn.Dropout(p=0.3)
        self.bnfc1 = nn.BatchNorm1d(n_channel)
        self.fc2 = nn.Linear(n_channel, n_output)
        
    def reset_weights(self):
        for m in self.modules():
            if hasattr(m, 'reset_parameters'):
                m.reset_parameters()

    def forward(self, x, age, print_shape=False):
        N = x.shape[0]
        
        # conv-bn-relu-pool 
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)

        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)

        if print_shape:
            print('Shape right before squeezing:', x.shape)

        x = self.final_pool(x).squeeze()
        if N == 1: x = x.reshape((1, -1))
        if self.use_age:
            x = torch.cat((x, age.reshape(-1, 1)), dim=1)

        # fc-bn-dropout-relu-fc
        x = self.fc1(x)
        x = self.bnfc1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        return x
        # return F.log_softmax(x, dim=1)

In [16]:
def generate_TinyCNN():
    return TinyCNN(n_input=train_dataset[0]['signal'].shape[0], 
                   n_output=3, use_age=True, final_pool='max')

In [17]:
model = generate_TinyCNN()
model = model.to(device, dtype=torch.float32)
print(model)
print()

# tensorboard visualization
# visualize_network_tensorboard(model, '1D-Tiny-CNN')

# number of parameters
n = count_parameters(model)
print(f'The Number of parameters of the model: {n:,}')
print()

# calculate the last shape right before squeezing
print_final_shape(model)

del model

TinyCNN(
  (conv1): Conv1d(20, 64, kernel_size=(35,), stride=(7,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 64, kernel_size=(7,), stride=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (final_pool): AdaptiveMaxPool1d(output_size=1)
  (fc1): Linear(in_features=65, out_features=64, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (bnfc1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)

The Number of parameters of the model: 78,403

Shape right before squeezing: torch.Size([32, 64, 32])


C:\Users\IPIS-Minjae\anaconda3\envs\EEG_Project\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


#### M5-like model

In [18]:
class M5(nn.Module):
    def __init__(self, in_channels=20, base_channels=256, 
                 out_dims=3, use_age=True, final_pool='max'):
        super().__init__()
        
        if final_pool not in {'average', 'max'}:
            raise ValueError("final_pool must be set to one of ['average', 'max']")
        
        self.use_age = use_age
        
        self.conv1 = nn.Conv1d(in_channels, base_channels, kernel_size=41, stride=2)
        self.bn1 = nn.BatchNorm1d(base_channels)
        self.pool1 = nn.MaxPool1d(2)
        
        self.conv2 = nn.Conv1d(base_channels, base_channels, kernel_size=11)
        self.bn2 = nn.BatchNorm1d(base_channels)
        self.pool2 = nn.MaxPool1d(2)
        
        self.conv3 = nn.Conv1d(base_channels, 2 * base_channels, kernel_size=11)
        self.bn3 = nn.BatchNorm1d(2 * base_channels)
        self.pool3 = nn.MaxPool1d(3)
        
        self.conv4 = nn.Conv1d(2 * base_channels, 2 * base_channels, kernel_size=11)
        self.bn4 = nn.BatchNorm1d(2 * base_channels)
        self.pool4 = nn.MaxPool1d(3)
        
        self.conv5 = nn.Conv1d(2 * base_channels, 2 * base_channels, kernel_size=11)
        self.bn5 = nn.BatchNorm1d(2 * base_channels)
        self.pool5 = nn.MaxPool1d(2)
        
        if final_pool == 'average':
            self.final_pool = nn.AdaptiveAvgPool1d(1)
        elif final_pool == 'max':
            self.final_pool = nn.AdaptiveMaxPool1d(1)
    
        if self.use_age:        
            self.fc1 = nn.Linear(2 * base_channels + 1, 2 * base_channels)
        else:
            self.fc1 = nn.Linear(2 * base_channels, 2 * base_channels)
            
        self.dropout = nn.Dropout(p=0.3)
        self.bnfc1 = nn.BatchNorm1d(2 * base_channels)
        self.fc2 = nn.Linear(2 * base_channels, out_dims)
        
    def reset_weights(self):
        for m in self.modules():
            if hasattr(m, 'reset_parameters'):
                m.reset_parameters()

    def forward(self, x, age, print_shape=False):
        N = x.shape[0]
        
        # conv-bn-relu-pool 
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)

        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)

        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)

        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)

        x = self.conv5(x)
        x = F.relu(self.bn5(x))
        x = self.pool5(x)

        if N == 1: x = x.reshape((1, -1))
        if print_shape:
            print('Shape right before squeezing:', x.shape)
    
        x = self.final_pool(x).squeeze()
        if self.use_age:
            x = torch.cat((x, age.reshape(-1, 1)), dim=1)

        # fc-bn-dropout-relu-fc
        x = self.fc1(x)
        x = self.bnfc1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        return x
        # return F.log_softmax(x, dim=1)

In [19]:
def generate_M5():
    return M5(in_channels=train_dataset[0]['signal'].shape[0], 
              out_dims=3, use_age=True, final_pool='max')

In [20]:
model = generate_M5()
model = model.to(device, dtype=torch.float32)
print(model)
print()

# tensorboard visualization
# visualize_network_tensorboard(model, 'M5')

# number of parameters
n = count_parameters(model)
print(f'The Number of parameters of the model: {n:,}')
print()

# calculate the last shape right before squeezing
print_final_shape(model)

del model

M5(
  (conv1): Conv1d(20, 256, kernel_size=(41,), stride=(2,))
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(256, 256, kernel_size=(11,), stride=(1,))
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(256, 512, kernel_size=(11,), stride=(1,))
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(512, 512, kernel_size=(11,), stride=(1,))
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv1d(512, 512, kernel_size=(11,), stride

#### M5-like model without the usage of age

In [21]:
def generate_M5_no_age():
    return M5(in_channels=train_dataset[0]['signal'].shape[0], 
              out_dims=3, use_age=False, final_pool='max')

In [22]:
model = generate_M5_no_age()
model = model.to(device, dtype=torch.float32)
print(model)
print()

# tensorboard visualization
# visualize_network_tensorboard(model, 'M5-no-age')

# number of parameters
n = count_parameters(model)
print(f'The Number of parameters of the model: {n:,}')
print()

# calculate the last shape right before squeezing
print_final_shape(model)

del model

M5(
  (conv1): Conv1d(20, 256, kernel_size=(41,), stride=(2,))
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(256, 256, kernel_size=(11,), stride=(1,))
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(256, 512, kernel_size=(11,), stride=(1,))
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(512, 512, kernel_size=(11,), stride=(1,))
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv1d(512, 512, kernel_size=(11,), stride

#### 1D ResNet model

In [23]:
class BasicResBlock(nn.Module):
    expansion: int = 1

    def __init__(self, c_in, c_out, kernel_size, stride, groups=1) -> None:
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=c_in, out_channels=c_out, 
                               kernel_size=kernel_size, stride=stride, 
                               padding=kernel_size//2, bias=False)
        self.bn1 = nn.BatchNorm1d(c_out)

        self.conv2 = nn.Conv1d(in_channels=c_out, out_channels=c_out, groups=groups,
                               kernel_size=kernel_size, stride=1, 
                               padding=kernel_size//2, bias=False)
        self.bn2 = nn.BatchNorm1d(c_out)

        self.relu = nn.ReLU(inplace=True)
        
        self.downsample = None
        if stride != 1 or c_in != c_out:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels=c_in, out_channels=c_out, 
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(c_out)
            )
                    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        if self.downsample is not None:
            identity = self.downsample(identity)
        x = self.relu(x + identity)
        
        return x

In [24]:
class BottleneckBlock(nn.Module):
    expansion: int = 4
        
    def __init__(self, c_in, c_out, kernel_size, stride, groups=1) -> None:
        super().__init__()
        width = c_out        
        self.conv1 = nn.Conv1d(in_channels=c_in, out_channels=width,
                               kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm1d(width)

        self.conv2 = nn.Conv1d(in_channels=width, out_channels=width, groups=groups,
                               kernel_size=kernel_size, stride=stride, 
                               padding=kernel_size//2, bias=False)
        self.bn2 = nn.BatchNorm1d(width)

        self.conv3 = nn.Conv1d(in_channels=width, out_channels=c_out*self.expansion, 
                               kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm1d(c_out*self.expansion)
        
        self.relu = nn.ReLU(inplace=True)
        
        self.downsample = None
        if stride != 1 or c_in != c_out*self.expansion:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels=c_in, out_channels=c_out*self.expansion, 
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(c_out*self.expansion)
            )
                    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        if self.downsample is not None:
            identity = self.downsample(identity)
            
        x = self.relu(x + identity)
        
        return x

In [25]:
class ResNet(nn.Module):
    def __init__(self, 
                 block: Type[Union[BasicResBlock, BottleneckBlock]], 
                 conv_layers: List[int],
                 fc_stages: int,
                 in_channels=20,
                 out_dims=3,
                 starting_channels=64,
                 groups=1,
                 kernel_size=9, 
                 use_age=True, 
                 final_pool='max') -> None:
        
        super().__init__()
        
        if final_pool not in {'average', 'max'}:
            raise ValueError("final_pool must be set to one of ['average', 'max']")        

        self.current_channels = starting_channels
        self.use_age = use_age
        self.groups = groups
        
        self.input_stage = nn.Sequential(
            nn.Conv1d(in_channels=in_channels, out_channels=starting_channels, 
                      kernel_size=kernel_size*3, stride=2,
                      padding=(kernel_size*3)//2, bias=False),
            nn.BatchNorm1d(starting_channels), 
            nn.ReLU(),
        )
        
        self.conv_stage1 = self._make_conv_layer(block, conv_layers[0], starting_channels, kernel_size, stride=3)
        self.conv_stage2 = self._make_conv_layer(block, conv_layers[1], starting_channels*2, kernel_size, stride=3)
        self.conv_stage3 = self._make_conv_layer(block, conv_layers[2], starting_channels*4, kernel_size, stride=3)
        self.conv_stage4 = self._make_conv_layer(block, conv_layers[3], starting_channels*8, kernel_size, stride=3)
        
        if final_pool == 'average':
            self.final_pool = nn.AdaptiveAvgPool1d(1)
        elif final_pool == 'max':
            self.final_pool = nn.AdaptiveMaxPool1d(1)
        
        fc_stage = []        
        if self.use_age:
            self.current_channels = self.current_channels + 1
        
        for l in range(fc_stages):
            layer = nn.Sequential(nn.Linear(self.current_channels, self.current_channels // 2, bias=False),
                                  nn.Dropout(p=0.1),
                                  nn.BatchNorm1d(self.current_channels // 2), 
                                  nn.ReLU())
            self.current_channels = self.current_channels // 2
            fc_stage.append(layer)
        fc_stage.append(nn.Linear(self.current_channels, out_dims))
        self.fc_stage = nn.Sequential(*fc_stage)
        
    def reset_weights(self):
        for m in self.modules():
            if hasattr(m, 'reset_parameters'):
                m.reset_parameters()
        
    def _make_conv_layer(self, block: Type[Union[BasicResBlock, BottleneckBlock]], 
                         n_block: int, c_out: int, kernel_size: int, stride: int = 1) -> nn.Sequential:
        layers = []
        c_in = self.current_channels
        layers.append(block(c_in, c_out, kernel_size, groups=self.groups, stride=1))

        c_in = c_out * block.expansion
        self.current_channels = c_in
        for _ in range(1, n_block):
            layers.append(block(c_in, c_out, kernel_size, groups=self.groups, stride=1))
            
        layers.append(nn.MaxPool1d(kernel_size=stride))
            
        return nn.Sequential(*layers)

    def forward(self, x, age, print_shape=False):
        N = x.shape[0]
        
        x = self.input_stage(x)
        
        x = self.conv_stage1(x)
        x = self.conv_stage2(x)
        x = self.conv_stage3(x)
        x = self.conv_stage4(x)

        if print_shape:
            print('Shape right before squeezing:', x.shape)
                
        x = self.final_pool(x).squeeze()
        if N == 1: x = x.reshape((1, -1))
        if self.use_age:
            x = torch.cat((x, age.reshape(-1, 1)), dim=1)
        x = self.fc_stage(x)
        
        return x
        # return F.log_softmax(x, dim=2)

In [26]:
def generate_ResNet():
    return ResNet(block=BottleneckBlock, 
                  conv_layers=[2, 2, 2, 2], 
                  fc_stages=3, 
                  in_channels=train_dataset[0]['signal'].shape[0], 
                  out_dims=3, 
                  starting_channels=64,
                  kernel_size=9, 
                  use_age=True, 
                  final_pool='max')

In [27]:
model = generate_ResNet()
model = model.to(device, dtype=torch.float32)
print(model)
print()

# tensorboard visualization
# visualize_network_tensorboard(model, '1D-ResNet')

# number of parameters
n = count_parameters(model)
print(f'The Number of parameters of the model: {n:,}')
print()

# calculate the last shape right before squeezing
print_final_shape(model)

del model

ResNet(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
      (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=Tru

#### 1D CNN: ResNet-like model without the usage of age

In [28]:
def generate_ResNet_no_age():
    return ResNet(block=BottleneckBlock, 
                  conv_layers=[2, 2, 2, 2], 
                  fc_stages=3, 
                  in_channels=train_dataset[0]['signal'].shape[0], 
                  out_dims=3, 
                  starting_channels=64,
                  kernel_size=9, 
                  use_age=False, 
                  final_pool='max')

In [29]:
model = generate_ResNet_no_age()
model = model.to(device, dtype=torch.float32)
print(model)
print()

# tensorboard visualization
# visualize_network_tensorboard(model, '1D-ResNet-no-age')

# number of parameters
n = count_parameters(model)
print(f'The Number of parameters of the model: {n:,}')
print()

# calculate the last shape right before squeezing
print_final_shape(model)

del model

ResNet(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
      (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=Tru

#### 1D-ResNet-50

In [30]:
def generate_ResNet50():
    return ResNet(block=BottleneckBlock, 
                  conv_layers=[3, 4, 6, 3], 
                  fc_stages=3, 
                  in_channels=train_dataset[0]['signal'].shape[0], 
                  out_dims=3, 
                  starting_channels=64,
                  kernel_size=9, 
                  use_age=True, 
                  final_pool='max')

In [31]:
model = generate_ResNet50()
model = model.to(device, dtype=torch.float32)
print(model)
print()

# tensorboard visualization
# visualize_network_tensorboard(model, '1D-Tiny-ResNet')

# number of parameters
n = count_parameters(model)
print(f'The Number of parameters of the model: {n:,}')
print()

# calculate the last shape right before squeezing
print_final_shape(model)

del model

ResNet(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
      (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=Tru

#### 1D Tiny ResNet

In [32]:
def generate_TinyResNet():
    return ResNet(block=BasicResBlock, 
                  conv_layers=[1, 1, 1, 1], 
                  fc_stages=3, 
                  in_channels=train_dataset[0]['signal'].shape[0], 
                  out_dims=3, 
                  starting_channels=64,
                  kernel_size=9, 
                  use_age=True, 
                  final_pool='max')

In [33]:
model = generate_TinyResNet()
model = model.to(device, dtype=torch.float32)
print(model)
print()

# tensorboard visualization
# visualize_network_tensorboard(model, '1D-Tiny-ResNet')

# number of parameters
n = count_parameters(model)
print(f'The Number of parameters of the model: {n:,}')
print()

# calculate the last shape right before squeezing
print_final_shape(model)

del model

ResNet(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BasicResBlock(
      (conv1): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_stage2): Sequential(
    (0): BasicResBlock(
      (conv1): Conv1d(64, 128, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

#### 1D Multi-Dilated ResNet

In [34]:
class MultiBottleneckBlock(nn.Module):
    expansion: int = 4
        
    def __init__(self, c_in, c_out, kernel_size, stride, groups=1) -> None:
        super().__init__()
        width = c_out
        
        self.conv1 = nn.Conv1d(in_channels=c_in, out_channels=width, kernel_size=1, 
                                 dilation=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm1d(width)

        self.conv2_1 = nn.Conv1d(in_channels=width, out_channels=width, groups=groups,
                                 kernel_size=kernel_size, dilation=1, 
                                 stride=stride, padding=kernel_size//2, bias=False)
        self.conv2_2 = nn.Conv1d(in_channels=width, out_channels=width, groups=groups,
                                 kernel_size=kernel_size, dilation=2, 
                                 stride=stride, padding=(kernel_size//2)*2, bias=False)
        self.bn2 = nn.BatchNorm1d(width*2)
        
        self.conv3 = nn.Conv1d(in_channels=width*2, out_channels=c_out*self.expansion, kernel_size=1, 
                                 dilation=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm1d(c_out*self.expansion)
                
        self.downsample = None
        if stride != 1 or c_in != c_out*self.expansion:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels=c_in, out_channels=c_out*self.expansion, 
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(c_out*self.expansion)
            )
                    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        
        x1 = self.conv2_1(x)
        x2 = self.conv2_2(x)
        x = torch.cat((x1, x2), dim=1)
        x = self.bn2(x)
        x = F.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        
        if self.downsample is not None:
            identity = self.downsample(identity)
            
        x = F.relu(x + identity)
        return x

In [35]:
def generate_Multi_Dilated_ResNet():
    return ResNet(block=MultiBottleneckBlock, 
                  conv_layers=[3, 4, 6, 3], 
                  fc_stages=3, 
                  in_channels=train_dataset[0]['signal'].shape[0], 
                  out_dims=3, 
                  starting_channels=32,
                  kernel_size=9, 
                  use_age=True, 
                  final_pool='max')

In [36]:
model = generate_Multi_Dilated_ResNet()
model = model.to(device, dtype=torch.float32)
print(model)
print()

# tensorboard visualization
# visualize_network_tensorboard(model, '1D-Multi-Dilated-ResNet')

# number of parameters
n = count_parameters(model)
print(f'The Number of parameters of the model: {n:,}')
print()

# calculate the last shape right before squeezing
print_final_shape(model)

del model

ResNet(
  (input_stage): Sequential(
    (0): Conv1d(20, 32, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): MultiBottleneckBlock(
      (conv1): Conv1d(32, 32, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_1): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (conv2_2): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(8,), dilation=(2,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv1d(64, 128, kernel_size=(1,), stride=(1,), bias=False)
      (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv1d(32, 128, kernel_size=(1,), stride=(1,),

#### ResNeXt-50

In [37]:
def generate_ResNeXt_50():
    return ResNet(block=BottleneckBlock, 
                  conv_layers=[3, 4, 7, 2], 
                  fc_stages=3, 
                  in_channels=train_dataset[0]['signal'].shape[0], 
                  out_dims=3, 
                  starting_channels=64,
                  groups=32,
                  kernel_size=9, 
                  use_age=True, 
                  final_pool='max')

In [38]:
model = generate_ResNeXt_50()
model = model.to(device, dtype=torch.float32)
print(model)
print()

# tensorboard visualization
# visualize_network_tensorboard(model, '1D-ResNet')

# number of parameters
n = count_parameters(model)
print(f'The Number of parameters of the model: {n:,}')
print()

# calculate the last shape right before squeezing
print_final_shape(model)

del model

ResNet(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), groups=32, bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
      (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1,

-----

## Some useful functions for training

In [39]:
def train_multistep(model, loader, optimizer, scheduler, config, steps):
    model.train()
        
    i = 0
    cumu_loss = 0
    correct, total = (0, 0)
    
    while True:
        for sample_batched in loader:
            optimizer.zero_grad()

            # load the mini-batched data
            x = sample_batched['signal'].to(device)
            age = sample_batched['age'].to(device)
            y = sample_batched['class_label'].to(device)
            
            # forward pass
            output = model(x, age)
            
            # loss function
            if config['criterion'] == 'cross-entropy':
                s = F.log_softmax(output, dim=1)
                loss = F.nll_loss(s, y)
            elif config['criterion'] == 'multi-bce':
                y_oh = F.one_hot(y, num_classes=len(class_label_to_type))
                s = torch.sigmoid(output)
                loss = F.binary_cross_entropy_with_logits(output, y_oh)

            # backward and update
            loss.backward()
            optimizer.step()
            scheduler.step()

            # train accuracy
            pred = s.argmax(dim=-1)
            correct += pred.squeeze().eq(y).sum().item()
            total += pred.shape[0]
            cumu_loss += loss.item()
            
            i += 1
            if steps <= i: break
        if steps <= i: break
            
    train_acc = 100.0 * correct / total
    avg_loss = cumu_loss / steps
    
    return (avg_loss, train_acc)


def train_mixup_multistep(model, loader, optimizer, scheduler, config, steps):
    model.train()
        
    i = 0
    cumu_loss = 0
    correct, total = (0, 0)
    
    while True:
        for sample_batched in loader:
            optimizer.zero_grad()

            # load and mixup the mini-batched data
            x1 = sample_batched['signal'].to(device)
            age1 = sample_batched['age'].to(device)
            y1 = sample_batched['class_label'].to(device)

            index = torch.randperm(x1.shape[0]).cuda()                
            x2 = x1[index]
            age2 = age1[index]
            y2 = y1[index]
            
            mixup_alpha = config['mixup']
            lam = np.random.beta(mixup_alpha, mixup_alpha)
            x = lam * x1 + (1.0 - lam) * x2
            age = lam * age1 + (1.0 - lam) * age2

            # forward pass
            output = model(x, age)
            
            # loss function
            if config['criterion'] == 'cross-entropy':
                s = F.log_softmax(output, dim=1)
                loss1 = F.nll_loss(s, y1)
                loss2 = F.nll_loss(s, y2)
                loss = lam * loss1 + (1 - lam) * loss2
            elif config['criterion'] == 'multi-bce':
                y1_oh = F.one_hot(y1, num_classes=len(class_label_to_type))
                y2_oh = F.one_hot(y2, num_classes=len(class_label_to_type))
                y_oh = lam * y1_oh + (1.0 - lam) * y2_oh
                s = torch.sigmoid(output)
                loss = F.binary_cross_entropy_with_logits(output, y_oh)

            # backward and update
            loss.backward()
            optimizer.step()
            scheduler.step()

            # train accuracy
            pred = s.argmax(dim=-1)
            correct1 = pred.squeeze().eq(y1).sum().item()
            correct2 = pred.squeeze().eq(y2).sum().item()
            correct += lam * correct1 + (1.0 - lam) * correct2
            total += pred.shape[0]
            cumu_loss += loss.item()
            
            i += 1
            if steps <= i: break
        if steps <= i: break
            
    train_acc = 100.0 * correct / total
    avg_loss = cumu_loss / steps
    
    return (avg_loss, train_acc)

In [40]:
def check_accuracy(model, loader, config, repeat=1):
    model.eval()
    
    # for test accuracy
    correct, total = (0, 0) 
    
    # for confusion matrix
    C = len(class_label_to_type)
    test_confusion = np.zeros((C, C), dtype=np.int32)
    
    # for test debug
    test_debug = {data['metadata']['serial']: 
                  {'GT': data['class_label'].item(), 
                   'Acc': 0, 
                   'Pred': [0] * C} for data in loader.dataset}
    
    # for ROC curve
    score = None
    target = None
    
    with torch.no_grad():
        for k in range(repeat):
            for sample_batched in loader:
                # pull up the data
                x = sample_batched['signal'].to(device)
                age = sample_batched['age'].to(device)
                y = sample_batched['class_label'].to(device)

                # apply model on whole batch directly on device
                output = model(x, age)
                
                if config['criterion'] == 'cross-entropy':
                    s = F.softmax(output, dim=1)
                elif config['criterion'] == 'multi-bce':
                    s = torch.sigmoid(output)
                
                # test accuracy
                pred = s.argmax(dim=-1)
                correct += pred.squeeze().eq(y).sum().item()
                total += pred.shape[0]

                if score is None:
                    score = s.detach().cpu().numpy()
                    target = y.detach().cpu().numpy()
                else:
                    score = np.concatenate((score, s.detach().cpu().numpy()), axis=0)
                    target = np.concatenate((target, y.detach().cpu().numpy()), axis=0)

                # confusion matrix
                test_confusion += calculate_confusion_matrix(pred, y)

                # test debug
                for n in range(pred.shape[0]):
                    serial = sample_batched['metadata'][n]['serial']
                    test_debug[serial]['edfname'] = sample_batched['metadata'][n]['edfname']
                    test_debug[serial]['Pred'][pred[n].item()] += 1
                    acc = test_debug[serial]['Pred'][y[n].item()] / np.sum(test_debug[serial]['Pred']) * 100
                    test_debug[serial]['Acc'] = f'{acc:>6.02f}%'
        
    test_accuracy = 100.0 * correct / total
    return (test_accuracy, test_confusion, test_debug, score, target)


def calculate_confusion_matrix(pred, target):
    N = target.shape[0]
    C = len(class_label_to_type)
    confusion = np.zeros((C, C), dtype=np.int32)
    
    for i in range(N):
        r = target[i]
        c = pred[i]
        confusion[r, c] += 1
    return confusion

In [41]:
def draw_loss_plot(losses, lr_decay_step):
    plt.style.use('default') # default, ggplot, fivethirtyeight, classic
    fig = plt.figure(num=1, clear=True, figsize=(8.0, 3.0), constrained_layout=True)
    ax = fig.add_subplot(1, 1, 1)
    
    N = len(losses)
    x = np.arange(1, N + 1)
    ax.plot(x, losses)
    
    if type(lr_decay_step) is list:
        ax.vlines(lr_decay_step, 0, 1, transform=ax.get_xaxis_transform(), 
                  colors='m', alpha=0.5, linestyle='solid')
    else:
        x2 = np.arange(lr_decay_step, N, lr_decay_step)
        ax.vlines(x2, 0, 1, transform=ax.get_xaxis_transform(), 
                  colors='m', alpha=0.5, linestyle='solid')
    # ax.vlines([1, N], 0, 1, transform=ax.get_xaxis_transform(), 
    #           colors='k', alpha=0.7, linestyle='solid')
    
    ax.set_xlim(left=0)
    ax.set_title('Loss Plot')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Training Loss')
    
    plt.show()
    fig.clear()
    plt.close(fig)

def draw_accuracy_history(train_acc_history, val_acc_history, history_interval, lr_decay_step):
    plt.style.use('default') # default, ggplot, fivethirtyeight, classic
    fig = plt.figure(num=1, clear=True, figsize=(8.0, 3.0), constrained_layout=True)
    ax = fig.add_subplot(1, 1, 1)
    
    N = len(train_acc_history) * history_interval
    x = np.arange(history_interval, N + 1, history_interval)
    ax.plot(x, train_acc_history, 'r-', label='Train accuracy')
    ax.plot(x, val_acc_history, 'b-', label='Validation accuracy')
    
    if type(lr_decay_step) is list:
        ax.vlines(lr_decay_step, 0, 1, transform=ax.get_xaxis_transform(), 
                  colors='m', alpha=0.5, linestyle='solid')
    else:
        x2 = np.arange(lr_decay_step, N + 1, lr_decay_step)
        ax.vlines(x2, 0, 1, transform=ax.get_xaxis_transform(), 
                  colors='m', alpha=0.5, linestyle='solid')
    # ax.vlines([history_interval, N], 0, 1, transform=ax.get_xaxis_transform(), 
    #           colors='k', alpha=0.7, linestyle='solid')
    
    ax.set_xlim(left=0)
    ax.legend(loc='lower right')
    ax.set_title('Accuracy Plot during Training')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Accuracy (%)')
    
    plt.show()
    fig.clear()
    plt.close(fig)

In [42]:
def draw_confusion(confusion):
    C = len(class_label_to_type)
    
    plt.style.use('default') # default, ggplot, fivethirtyeight, classic
    plt.rcParams['image.cmap'] = 'jet' # 'nipy_spectral'

    fig = plt.figure(num=1, clear=True, figsize=(4.0, 4.0), constrained_layout=True)
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(confusion, alpha=0.8)

    ax.set_xticks(np.arange(C))
    ax.set_yticks(np.arange(C))
    ax.set_xticklabels(class_label_to_type)
    ax.set_yticklabels(class_label_to_type)
    
    for r in range(C):
        for c in range(C):
            text = ax.text(c, r, confusion[r, c],
                           ha="center", va="center", color='k')
    
    ax.set_title('Confusion Matrix')
    ax.set_xlabel('Prediction')
    ax.set_ylabel('Ground Truth')
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    # plt.show()
    wandb.log({'Confusion Matrix (Image)': wandb.Image(plt)})
    
    fig.clear()
    plt.close(fig)
    

def draw_roc_curve(score, target):
    plt.style.use('default') # default, ggplot, fivethirtyeight, classic
    
    # Binarize the output
    n_classes = len(class_label_to_type)
    target = label_binarize(target, classes=np.arange(n_classes))
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(target[:, i], score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(target.ravel(), score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    # aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    # draw class-agnostic ROC curve
    fig = plt.figure(num=1, clear=True, figsize=(8.5, 4.0), constrained_layout=True)
    ax = fig.add_subplot(1, 2, 1)
    lw = 1.5
    colors = cycle(['limegreen', 'mediumpurple', 'darkorange', 
                    'dodgerblue', 'lightcoral', 'goldenrod', 
                    'indigo', 'darkgreen', 'navy', 'brown'])
    for i, color in zip(range(n_classes), colors):
        ax.plot(fpr[i], tpr[i], color=color, lw=lw,
                label='{0} (area = {1:0.2f})'
                ''.format(class_label_to_type[i], roc_auc[i]))    
    ax.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Class-Wise ROC Curves')
    ax.legend(loc="lower right")

    # Plot class-aware ROC curves
    ax = fig.add_subplot(1, 2, 2)
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle='-', linewidth=lw)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle='-', linewidth=lw)

    ax.plot([0, 1], [0, 1], 'k--', lw=lw)
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Class-Agnostic ROC Curves')
    ax.legend(loc="lower right")
    
    # plt.show()
    wandb.log({'ROC Curve': wandb.Image(plt)})
    
    fig.clear()
    plt.close(fig)

In [43]:
def learning_rate_search(model, min_log_lr, max_log_lr, trials, iters):
    learning_rate_record = []
    for t in tqdm(range(trials)):
        log_lr = np.random.uniform(min_log_lr, max_log_lr)
        lr = 10 ** log_lr
        
        model.reset_weights()
        model.train()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-1)
        correct, total = (0, 0)
        
        i = 1
        while True:
            for sample_batched in train_loader:
                x = sample_batched['signal'].to(device)
                age = sample_batched['age'].to(device)
                y = sample_batched['class_label'].to(device)

                output = model(x, age)
                pred = F.log_softmax(output, dim=1)
                loss = F.nll_loss(pred, y)

                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                pred = pred.argmax(dim=-1)
                correct += pred.squeeze().eq(y).sum().item()
                total += pred.shape[0]
                
                i += 1
                if i >= iters:
                    break
            if i >= iters:
                break
        
        train_accuracy = 100.0 * correct / total
        
        # Train accuracy for the final epoch is stored
        learning_rate_record.append((log_lr, train_accuracy))
    
    return learning_rate_record


def draw_learning_rate_record(learning_rate_record):
    plt.style.use('default') # default, ggplot, fivethirtyeight, classic

    fig = plt.figure(num=1, clear=True, constrained_layout=True) # figsize=(6.0, 6.0)
    ax = fig.add_subplot(1, 1, 1)
    
    ax.set_title('Learning Rate Search')
    ax.set_xlabel('Learning rate in log-scale')
    ax.set_ylabel('Train accuracy')

    ax.scatter(*max(learning_rate_record, key=lambda x: x[1]), 
               s=150, c='w', marker='o', edgecolors='limegreen')
    
    for log_lr, val_accuracy in learning_rate_record:
        ax.scatter(log_lr, val_accuracy, c='r',
                   alpha=0.5, edgecolors='none')
        
    plt.show()
    fig.clear()
    plt.close(fig)

-----

## Train models

In [44]:
model_pool = []

# model_dict = {}
# model_dict['name'] = '1D-Tiny-CNN'
# model_dict['generator'] = generate_TinyCNN
# model_dict['lr_start'] = None
# model_pool.append(model_dict)

model_dict = {}
model_dict['name'] = 'M5'
model_dict['generator'] = generate_M5
model_dict['lr_start'] = 1e-3
model_pool.append(model_dict)

# model_dict = {}
# model_dict['name'] = 'M5-no-age'
# model_dict['generator'] = generate_M5_no_age
# model_dict['lr_start'] = None
# model_pool.append(model_dict)

model_dict = {}
model_dict['name'] = '1D-ResNet-18'
model_dict['generator'] = generate_ResNet
model_dict['lr_start'] = 1e-3
model_pool.append(model_dict)

# model_dict = {}
# model_dict['name'] = '1D-ResNet-no-age'
# model_dict['generator'] = generate_ResNet_no_age
# model_dict['lr_start'] = None
# model_pool.append(model_dict)

# model_dict = {}
# model_dict['name'] = '1D-ResNet-50'
# model_dict['generator'] = generate_ResNet50
# model_dict['lr_start'] = None
# model_pool.append(model_dict)


# model_dict = {}
# model_dict['name'] = '1D-Tiny-ResNet'
# model_dict['generator'] = generate_TinyResNet
# model_dict['lr_start'] = None
# model_pool.append(model_dict)

# model_dict = {}
# model_dict['name'] = '1D-Multi-Dilated-ResNet'
# model_dict['generator'] = generate_Multi_Dilated_ResNet
# model_dict['lr_start'] = None
# model_pool.append(model_dict)

model_dict = {}
model_dict['name'] = '1D-ResNeXt-50'
model_dict['generator'] = generate_ResNeXt_50
model_dict['lr_start'] = 1e-3
model_pool.append(model_dict)

pprint.pp(model_pool, width=150)

[{'name': 'M5', 'generator': <function generate_M5 at 0x0000016592DF9430>, 'lr_start': 0.001},
 {'name': '1D-ResNet-18', 'generator': <function generate_ResNet at 0x0000016592E063A0>, 'lr_start': 0.001},
 {'name': '1D-ResNeXt-50', 'generator': <function generate_ResNeXt_50 at 0x0000016592E2CF70>, 'lr_start': 0.001}]


In [45]:
for model_dict in model_pool:
    if model_dict['lr_start'] is None:
        print(f'{model_dict["name"]} LR searching..')
        model = model_dict['generator']().to(device)
        model.train()
        
        record = learning_rate_search(model, min_log_lr=-4.5, max_log_lr=-1.4, 
                                      trials=500, iters=30)
        
        draw_learning_rate_record(record)
        best_log_lr = record[np.argmax(np.array([v for lr, v in record]))][0]
        model_dict['lr_start'] = 10 ** best_log_lr
        
        print(f'best lr {model_dict["lr_start"]:.5e} / log_lr {best_log_lr}')
    else:
        print(f'{model_dict["name"]}: {model_dict["lr_start"]:.5e}')
        
    print('-' * 100)
        
pprint.pp(model_pool, width=150)

M5: 1.00000e-03
----------------------------------------------------------------------------------------------------
1D-ResNet-18: 1.00000e-03
----------------------------------------------------------------------------------------------------
1D-ResNeXt-50: 1.00000e-03
----------------------------------------------------------------------------------------------------
[{'name': 'M5', 'generator': <function generate_M5 at 0x0000016592DF9430>, 'lr_start': 0.001},
 {'name': '1D-ResNet-18', 'generator': <function generate_ResNet at 0x0000016592E063A0>, 'lr_start': 0.001},
 {'name': '1D-ResNeXt-50', 'generator': <function generate_ResNeXt_50 at 0x0000016592E2CF70>, 'lr_start': 0.001}]


In [47]:
file_check = True
save_model = True
save_temporary = False
draw_result = True

# log path
log_path = f'history_temp/{nb_fname}/'
os.makedirs(log_path, exist_ok=True)

# base configuration
base_cfg = {}
base_cfg['crop_length'] = crop_length
base_cfg['input_norm'] = 'dataset' # 'datatset', datapoint', 'no'
base_cfg['iterations'] = 100000
base_cfg['history_interval'] = base_cfg['iterations'] // 500
base_cfg['lr_decay_step'] = round(base_cfg['iterations'] * 0.8)
base_cfg['weight_decay'] = 1e-2
base_cfg['awgn'] = 5e-2
base_cfg['mixup'] = 0.3
base_cfg['criterion'] = 'multi-bce' # 'cross-entropy', 'multi-bce'

tr_ms = train_multistep if base_cfg.get('mixup', 0) < 1e-3 else train_mixup_multistep
    
# progress bar
pbar = tqdm(total=len(model_pool) * base_cfg['iterations'])

# train process on model_pool
for model_dict in model_pool:
    if file_check:
        path = os.path.join(log_path, f'{model_dict["name"]}-log')
        if os.path.isfile(path):
            log_dict = torch.load(path)
            if draw_result:
                draw_loss_plot(log_dict["losses"], log_dict["lr_decay_step"])
                draw_accuracy_history(log_dict["train_acc_history"], log_dict["val_acc_history"], 
                                      log_dict["history_interval"], log_dict["lr_decay_step"])            
            script = f'- train accuracy {log_dict["train_acc_history"][-1]:.2f}%, '\
            f'best / last test accuracies {log_dict["best_test_accuracy"]:.2f}% / {log_dict["last_test_accuracy"]:.2f}% - file exists'
            print('\n' + script + '\n')
            pbar.update(base_cfg['iterations'])
            continue
    
    print(f'{"*"*40} {model_dict["name"]} train starts {"*"*40}')
    
    # load the model dict
    model = model_dict['generator']().to(device)
    optimizer = optim.AdamW(model.parameters(), 
                            lr=model_dict['lr_start'], 
                            weight_decay=base_cfg['weight_decay'])
    scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                          step_size=base_cfg['lr_decay_step'], 
                                          gamma=0.1)
    
    # wandb initialization
    config = {}
    config.update(base_cfg)
    config['model'] = model_dict['name']
    config['num_params'] = count_parameters(model)
    config['LR'] = model_dict['lr_start']
    
    wandb_run = wandb.init(project="eeg-analysis", 
                           entity="ipis-mjkim", 
                           reinit=True,
                           save_code=True, 
                           name=nb_fname[9:] + '_' + model_dict["name"], 
                           config=config)

    with wandb_run:
        # train and validation routine
        best_val_acc = 0
        for i in range(0, wandb.config.iterations, wandb.config.history_interval):
            # train 'history_interval' steps
            loss, train_acc = tr_ms(model, train_loader, optimizer, scheduler, 
                                    wandb.config, wandb.config.history_interval)
            
            # validation
            val_acc, _, _, _, _ = check_accuracy(model, val_loader, config, repeat=10)
            
            if best_val_acc < val_acc:
                best_val_acc = val_acc
                best_model_state = deepcopy(model.state_dict())                
                if save_model and save_temporary:
                    path = os.path.join(log_path, f'{model_dict["name"]}')
                    torch.save(best_model_state, path)                    
                
            # log
            wandb.log({'Loss': loss, 
                       'Train Accuracy': train_acc, 
                       'Validation Accuracy': val_acc}, step=i)
            pbar.update(wandb.config.history_interval)

        # calculate the test accuracies for best and last models
        last_model_state = deepcopy(model.state_dict())
        last_test_result = check_accuracy(model, test_loader, wandb.config, repeat=30)
        last_test_acc = last_test_result[0]
        
        model.load_state_dict(best_model_state)
        best_test_result = check_accuracy(model, test_loader, wandb.config, repeat=30)
        best_test_acc = best_test_result[0]
 
        if last_test_acc < best_test_acc:
            model_state = best_model_state
            test_result = best_test_result
        else:
            model_state = last_model_state
            test_result = last_test_result
            
        model.load_state_dict(model_state)
        test_acc, test_confusion, test_debug, score, target = test_result
        
        # calculate the test accuracies for final model on much longer sequence
        last_test_result = check_accuracy(model, longer_test_loader, wandb.config, repeat=30)
        longer_test_acc = last_test_result[0]
        
        # save the model
        if save_model:
            path = os.path.join(log_path, f'{model_dict["name"]}')
            torch.save(model_state, path)
            
        # leave the message
        wandb.log({'Test Accuracy': test_acc,
                   '(Best / Last) Test Accuracy': ('Best' if last_test_acc < best_test_acc else 'Last', 
                                                   round(best_test_acc, 2), round(last_test_acc, 2)),
                   'Confusion Matrix (Array)': test_confusion,
                   'Test Debug Table': test_debug,
                   'Test Accuracy (Longer)': longer_test_acc})
        
        if draw_result:
            draw_roc_curve(score, target)
            draw_confusion(test_confusion)
            
    print('\n' + '-' * 100)

  0%|          | 0/300 [00:00<?, ?it/s]

**************************************** M5 train starts ****************************************


Loss,▁▁
Test Accuracy,▁
Test Accuracy (Longer),▁
Train Accuracy,▁█
Validation Accuracy,█▁
Loss,0.0
Test Accuracy,51.73077
Test Accuracy (Longer),52.37179
Train Accuracy,44.27666
Validation Accuracy,45.48077



----------------------------------------------------------------------------------------------------
**************************************** 1D-ResNet-18 train starts ****************************************


Loss,▁▁
Test Accuracy,▁
Test Accuracy (Longer),▁
Train Accuracy,▁█
Validation Accuracy,█▁
Loss,0.0
Test Accuracy,45.19231
Test Accuracy (Longer),45.19231
Train Accuracy,44.17223
Validation Accuracy,44.23077



----------------------------------------------------------------------------------------------------
**************************************** 1D-ResNeXt-50 train starts ****************************************


Loss,▁▁
Test Accuracy,▁
Test Accuracy (Longer),▁
Train Accuracy,▁█
Validation Accuracy,▁▁
Loss,0.0
Test Accuracy,44.96795
Test Accuracy (Longer),45.0641
Train Accuracy,45.33939
Validation Accuracy,44.42308



----------------------------------------------------------------------------------------------------
